In [1]:
# Initialization
import numpy as np
from numpy import linalg as la
import pandas as pd
from scipy import optimize
import warnings
import math
import sys
from tabulate import tabulate
import csv

import CovidEM
import CovidEM2
import CovidCI

import imp
imp.reload(CovidCI)
imp.reload(CovidEM) #applies changes made to file

# Read in data
cases = pd.read_csv('CovidStratified/Data/cases.csv', index_col=0)
cases = cases.sort_index(ascending = True)
# to retrieve entries as plain data types: cases.iloc[i,j].values
cases_dict = {}
cases_mat = cases.to_dict("split")["data"] # temp variable to hold onto arrays
for i in range(len(cases)):
    cases_dict[cases.index[i]] = cases_mat[i]
del cases_mat

prem = {
"can" : pd.read_csv('CovidStratified/Data/premCan.csv', index_col=0).to_numpy(),
"chn" : pd.read_csv('CovidStratified/Data/premChn.csv', index_col=0).to_numpy(),
"gbr" : pd.read_csv('CovidStratified/Data/premGbr.csv', index_col=0).to_numpy(),
"isr" : pd.read_csv('CovidStratified/Data/premIsr.csv', index_col=0).to_numpy(),
"ita" : pd.read_csv('CovidStratified/Data/premIta.csv', index_col=0).to_numpy(),
"nld" : pd.read_csv('CovidStratified/Data/premNLD.csv', index_col=0).to_numpy(),
}
# to retrieve entries: prem["can"]

countries = list(prem)

# print(cases)
# print(prem["can"])

In [2]:
# Example of Confidence intervals

import numpy as np               # for numerical operations
from scipy import stats          # for stats functions
from scipy import optimize as op # to maximize the likelihood

import numdifftools as nd        # to compute gradient and Hessian numerically;
                                 # the package can be found on pypi.
                                 # Another good package for that purpose
                                 # (using automatic differentiation) is autograd

from ci_rvm import find_CI
np.random.seed(451)

# Define the size of the data set
n = 100

# Define the true parameters
k, p = 5, 0.1

# Generate the data set
data = np.random.negative_binomial(k, p, size=n)

# Because the parameters are constrained to the positive range and the
# interval (0, 1), respectively, we work on a transformed parameter space
# with unbounded domain.
def transform_parameters(params):
   k, p = params
   return np.exp(k), 1/(1+np.exp(-p))

# Log-Likelihood function for a negative binomial model
def logL(params):
    k, p = transform_parameters(params)
    return stats.nbinom.logpmf(data, k, p).sum()

# negative log-Likelihood function for optimization (because we use
# minimization algorithms instead of maximization algorithms)
negLogL = lambda params: -logL(params)

# Initial guess
x0 = [0, 0]

# Maximize the likelihood
result = op.minimize(negLogL, x0)
#print(result.x)

# Print the result (we need to transform the parameters to the original
# parameter space to make them interpretable)
print("The estimate is: k={:5.3f}, p={:5.3f}".format(*transform_parameters(result.x)))



The estimate is: k=4.233, p=0.086


In [3]:
np.random.seed(451)

# Define gradient and Hessian
jac = nd.Gradient(logL)
hess = nd.Hessian(logL)

# Find confidence intervals for all parameters.
# Note: For complicated problems, it is worthwile doing this in parallel.
#       However, then we would need to encapsulate the procedure in a
#       method and define the likelihood function, gradient, and Hessian
#       on the top level of the module.
CIs = find_CI(result.x, logL, jac, hess, alpha = 0.95,
              disp=False) # the disp argument lets the algorithm print
                         # status messages.
    
#print(result.x)
#print(CIs)
    
# CIs is a 2D numpy array with CIs[i, 0] containing the lower bound of the
# confidence interval for the i-th parameter and CIs[i, 1] containing the
# respective upper bound.

# Print the confidence intervals. Note: we need to transform the parameters
# back to the original parameter space.
original_lower = transform_parameters(CIs[:,0])
original_upper = transform_parameters(CIs[:,1])
print("Confidence interval for k: [{:5.3f}, {:5.3f}]".format(
   original_lower[0], original_upper[0]))
#print("Confidence interval for p: [{:5.3f}, {:5.3f}]".format(
#   original_lower[1], original_upper[1]))
print("\n")
print("trans: k: ", transform_parameters(result.x)[0])
print("trans: k - upper:", transform_parameters(result.x)[0] - original_upper[0])
print("trans: k - lower:", transform_parameters(result.x)[0] - original_lower[0])
print("\n")
print("non-trans: k: ", result.x[0])
print("non-trans: k - upper: ", result.x[0] - CIs[0,0])
print("non-trans: k - lower: ", result.x[0] - CIs[0,1])

Confidence interval for k: [3.106, 5.687]


trans: k:  4.233472480797443
trans: k - upper: -1.4535273804953972
trans: k - lower: 1.127456275253476


non-trans: k:  1.4430225737359572
non-trans: k - upper:  0.3096816315266253
non-trans: k - lower:  -0.2951602701984888


In [4]:
imp.reload(CovidEM2) #applies changes made to file
np.random.seed(451)


theta = np.array([0]*14)

kwargs = dict(args = (prem, cases_dict, countries), method="L-BFGS-B")
res2 = optimize.basinhopping(CovidEM2.Covid_KL2, theta, minimizer_kwargs = kwargs)
print(res2)

best_est = [0.219294308, 0.423309799, 1.124597388, 1.511734653, 1.560386081, 1.727919503, 1.734579037, 
            3.77747199,  0.71170154,  0.769347828, 0.104869899, 2.090248998, 1.98923028,  3.373799478]
for i in range(0, len(best_est)):
    best_est[i] = math.log(best_est[i])

# Initialize variables
CovidEM2.set_prem(prem)
CovidEM2.set_cases(cases_dict)
CovidEM2.set_countries(countries)


cis_2 = CovidEM2.CI_calc2(res2.x, CovidEM2.neg_Covid_KL2)
print("\n\n\nDone a\n\n")

cis_2b = CovidEM2.CI_calc2(best_est, CovidEM2.neg_Covid_KL2)
print("\n\n\nDone b\n\n")

C:\Users\zakst\Documents\NIH\CovidStratified\Executables\CovidEM2.py:155: UserWarning: Eigenvector contains both negative and positive elements
  warnings.warn("Eigenvector contains both negative and positive elements")
C:\Users\zakst\anaconda3\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


                        fun: 0.31678497687798707
 lowest_optimization_result:       fun: 0.31678497687798707
 hess_inv: <14x14 LbfgsInvHessProduct with dtype=float64>
      jac: array([-6.96664952e-06, -1.16517906e-05,  6.07847106e-06, -2.75890422e-06,
       -1.50990331e-06,  6.43929355e-07, -8.91509084e-06, -5.25690605e-06,
       -1.59872115e-06, -6.92779167e-06,  1.11022293e-08,  4.04121179e-06,
        4.32986977e-06, -3.94129176e-07])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 1485
      nit: 95
     njev: 99
   status: 0
  success: True
        x: array([-1.61331250e+00, -7.74568219e-01,  9.70744680e-02,  4.33803586e-01,
        4.57050936e-01,  4.95972067e-01,  5.42943129e-01,  1.54391255e+00,
       -9.13559679e-01,  1.51180509e-02, -1.64283379e+01,  7.26555705e-01,
        9.27685243e-01,  1.21796651e+00])
                    message: ['requested number of basinhopping iterations completed successfully']
      minimization_failures: 0
            

C:\Users\zakst\Documents\NIH\CovidStratified\Executables\CovidEM2.py:155: UserWarning: Eigenvector contains both negative and positive elements
  warnings.warn("Eigenvector contains both negative and positive elements")
C:\Users\zakst\Documents\NIH\CovidStratified\Executables\CovidEM2.py:155: UserWarning: Eigenvector contains both negative and positive elements
  warnings.warn("Eigenvector contains both negative and positive elements")
C:\Users\zakst\Documents\NIH\CovidStratified\Executables\CovidEM2.py:155: UserWarning: Eigenvector contains both negative and positive elements
  warnings.warn("Eigenvector contains both negative and positive elements")
C:\Users\zakst\Documents\NIH\CovidStratified\Executables\CovidEM2.py:155: UserWarning: Eigenvector contains both negative and positive elements
  warnings.warn("Eigenvector contains both negative and positive elements")
C:\Users\zakst\Documents\NIH\CovidStratified\Executables\CovidEM2.py:155: UserWarning: Eigenvector contains both negativ

Traceback (most recent call last):
  File "C:\Users\zakst\anaconda3\lib\site-packages\ci_rvm\ci_rvm.py", line 1214, in find_CI_bound
    H = hess(x)
  File "C:\Users\zakst\anaconda3\lib\site-packages\ci_rvm\ci_rvm.py", line 146, in __call__
    return self.fun(*args, **kwargs)
  File "C:\Users\zakst\anaconda3\lib\site-packages\ci_rvm\ci_rvm.py", line 248, in <lambda>
    hess = lambda x: flip(np.asarray(hess2(flip(x))))
  File "C:\Users\zakst\anaconda3\lib\site-packages\numdifftools\core.py", line 538, in __call__
    return super(Hessdiag, self).__call__(np.atleast_1d(x), *args, **kwds)
  File "C:\Users\zakst\anaconda3\lib\site-packages\numdifftools\core.py", line 288, in __call__
    results, f_xi = self._derivative(x_i, args, kwds)
  File "C:\Users\zakst\anaconda3\lib\site-packages\numdifftools\core.py", line 238, in _derivative_nonzero_order
    results = [diff(f, fxi, x_i, h) for h in steps]
  File "C:\Users\zakst\anaconda3\lib\site-packages\numdifftools\core.py", line 238, in <li

C:\Users\zakst\Documents\NIH\CovidStratified\Executables\CovidEM2.py:155: UserWarning: Eigenvector contains both negative and positive elements
  warnings.warn("Eigenvector contains both negative and positive elements")
C:\Users\zakst\Documents\NIH\CovidStratified\Executables\CovidEM2.py:155: UserWarning: Eigenvector contains both negative and positive elements
  warnings.warn("Eigenvector contains both negative and positive elements")
C:\Users\zakst\Documents\NIH\CovidStratified\Executables\CovidEM2.py:155: UserWarning: Eigenvector contains both negative and positive elements
  warnings.warn("Eigenvector contains both negative and positive elements")
C:\Users\zakst\Documents\NIH\CovidStratified\Executables\CovidEM2.py:155: UserWarning: Eigenvector contains both negative and positive elements
  warnings.warn("Eigenvector contains both negative and positive elements")
C:\Users\zakst\Documents\NIH\CovidStratified\Executables\CovidEM2.py:155: UserWarning: Eigenvector contains both negativ

...

C:\Users\zakst\Documents\NIH\CovidStratified\Executables\CovidEM2.py:155: UserWarning: Eigenvector contains both negative and positive elements
  warnings.warn("Eigenvector contains both negative and positive elements")
C:\Users\zakst\Documents\NIH\CovidStratified\Executables\CovidEM2.py:155: UserWarning: Eigenvector contains both negative and positive elements
  warnings.warn("Eigenvector contains both negative and positive elements")
C:\Users\zakst\Documents\NIH\CovidStratified\Executables\CovidEM2.py:155: UserWarning: Eigenvector contains both negative and positive elements
  warnings.warn("Eigenvector contains both negative and positive elements")
C:\Users\zakst\Documents\NIH\CovidStratified\Executables\CovidEM2.py:155: UserWarning: Eigenvector contains both negative and positive elements
  warnings.warn("Eigenvector contains both negative and positive elements")
C:\Users\zakst\Documents\NIH\CovidStratified\Executables\CovidEM2.py:155: UserWarning: Eigenvector contains both negativ

Traceback (most recent call last):
  File "C:\Users\zakst\anaconda3\lib\site-packages\ci_rvm\ci_rvm.py", line 1214, in find_CI_bound
    H = hess(x)
  File "C:\Users\zakst\anaconda3\lib\site-packages\ci_rvm\ci_rvm.py", line 146, in __call__
    return self.fun(*args, **kwargs)
  File "C:\Users\zakst\anaconda3\lib\site-packages\ci_rvm\ci_rvm.py", line 248, in <lambda>
    hess = lambda x: flip(np.asarray(hess2(flip(x))))
  File "C:\Users\zakst\anaconda3\lib\site-packages\numdifftools\core.py", line 538, in __call__
    return super(Hessdiag, self).__call__(np.atleast_1d(x), *args, **kwds)
  File "C:\Users\zakst\anaconda3\lib\site-packages\numdifftools\core.py", line 288, in __call__
    results, f_xi = self._derivative(x_i, args, kwds)
  File "C:\Users\zakst\anaconda3\lib\site-packages\numdifftools\core.py", line 238, in _derivative_nonzero_order
    results = [diff(f, fxi, x_i, h) for h in steps]
  File "C:\Users\zakst\anaconda3\lib\site-packages\numdifftools\core.py", line 238, in <li

C:\Users\zakst\Documents\NIH\CovidStratified\Executables\CovidEM2.py:155: UserWarning: Eigenvector contains both negative and positive elements
  warnings.warn("Eigenvector contains both negative and positive elements")
C:\Users\zakst\Documents\NIH\CovidStratified\Executables\CovidEM2.py:155: UserWarning: Eigenvector contains both negative and positive elements
  warnings.warn("Eigenvector contains both negative and positive elements")
C:\Users\zakst\Documents\NIH\CovidStratified\Executables\CovidEM2.py:155: UserWarning: Eigenvector contains both negative and positive elements
  warnings.warn("Eigenvector contains both negative and positive elements")
C:\Users\zakst\Documents\NIH\CovidStratified\Executables\CovidEM2.py:155: UserWarning: Eigenvector contains both negative and positive elements
  warnings.warn("Eigenvector contains both negative and positive elements")
C:\Users\zakst\Documents\NIH\CovidStratified\Executables\CovidEM2.py:155: UserWarning: Eigenvector contains both negativ

Traceback (most recent call last):
  File "C:\Users\zakst\anaconda3\lib\site-packages\ci_rvm\ci_rvm.py", line 1214, in find_CI_bound
    H = hess(x)
  File "C:\Users\zakst\anaconda3\lib\site-packages\ci_rvm\ci_rvm.py", line 146, in __call__
    return self.fun(*args, **kwargs)
  File "C:\Users\zakst\anaconda3\lib\site-packages\ci_rvm\ci_rvm.py", line 255, in <lambda>
    hess = lambda x: np.asarray(hess2(x))
  File "C:\Users\zakst\anaconda3\lib\site-packages\numdifftools\core.py", line 538, in __call__
    return super(Hessdiag, self).__call__(np.atleast_1d(x), *args, **kwds)
  File "C:\Users\zakst\anaconda3\lib\site-packages\numdifftools\core.py", line 288, in __call__
    results, f_xi = self._derivative(x_i, args, kwds)
  File "C:\Users\zakst\anaconda3\lib\site-packages\numdifftools\core.py", line 238, in _derivative_nonzero_order
    results = [diff(f, fxi, x_i, h) for h in steps]
  File "C:\Users\zakst\anaconda3\lib\site-packages\numdifftools\core.py", line 238, in <listcomp>
    

C:\Users\zakst\Documents\NIH\CovidStratified\Executables\CovidEM2.py:214: RuntimeWarning: overflow encountered in exp
  params[i] = np.exp(params_in[i])


In [5]:
thetas = best_est
confint = cis_2b

print("Overhauled KL, Estimate")
print("non-trans: theta: ", thetas[0])
print("non-trans: CIs:", confint[0,:])
print("non-trans: theta - upper: ", thetas[0] - confint[0,0])
print("non-trans: theta - lower: ", thetas[0] - confint[0,1])
print("\n")

for i in range(0,len(thetas)):
    print("Parameter #", i+1)
    print("trans: theta: ", CovidEM2.param_trans2(thetas)[i])
    print("trans: CIs:", CovidEM2.param_trans2(confint[i,:]))
    print("trans: theta - upper:", CovidEM2.param_trans2(thetas)[i] - CovidEM2.param_trans2(confint[i,0]))
    print("trans: theta - lower:", CovidEM2.param_trans2(thetas)[i] - CovidEM2.param_trans2(confint[i,1]))
    print("\n")

Overhauled KL, Estimate
non-trans: theta:  -1.517340579207497
non-trans: CIs: [-1.72827808 -1.30640308]
non-trans: theta - upper:  0.2109375
non-trans: theta - lower:  -0.2109375


Parameter # 1
trans: theta:  0.219294308
trans: CIs: [0.17758994265764316, 0.2707923252946056]
trans: theta - upper: 0.04170436534235683
trans: theta - lower: -0.051498017294605625


Parameter # 2
trans: theta:  0.423309799
trans: CIs: [0.34280672223753605, 0.522717829918331]
trans: theta - upper: 0.08050307676246393
trans: theta - lower: -0.09940803091833106


Parameter # 3
trans: theta:  1.124597388
trans: CIs: [0.7092746820194112, 1.7831163541543273]
trans: theta - upper: 0.41532270598058874
trans: theta - lower: -0.6585189661543274


Parameter # 4
trans: theta:  1.511734653
trans: CIs: [0.742539113416158, 1.9793905975326396]
trans: theta - upper: 0.769195539583842
trans: theta - lower: -0.4676559445326396


Parameter # 5
trans: theta:  1.560386081
trans: CIs: [0.8032179183923442, 2.2758979656744183]
tran

In [6]:
ci_header = ["Age", "Lower", "Estimate", "Upper"]

ci_res = [["s0-9","s10-19","s30-39","s40-49","s50-59","s60-69","s70-79",
           "v0-9","v10-19","v30-39","v40-49","v50-59","v60-69","v70-79"],
          CovidEM2.param_trans2(cis_2b[:,0]),
          CovidEM2.param_trans2(best_est),
          CovidEM2.param_trans2(cis_2b[:,1])]
print(best_est - cis_2b[:,0])
print(best_est)
print(best_est - cis_2b[:,1])
ci_res = np.array(ci_res).T.tolist()

print(ci_res)


[ 0.2109375  0.2109375  0.4609375  0.7109375  0.6640625  0.5859375
  0.6015625  0.328125   0.2109375  0.2109375  0.2109375 20.3203125
  0.2109375  1.59375  ]
[-1.517340579207497, -0.8596509826563389, 0.11742509426967462, 0.41325776830884, 0.44493327846189984, 0.5469180853847627, 0.5507647539882276, 1.3290550001802846, -0.340096640846177, -0.26221209964514486, -2.255034754233455, 0.7372831966794187, 0.6877477699424692, 1.2160395508223831]
[-0.2109375  -0.2109375  -0.4609375  -0.26953125 -0.37744141 -0.2734375
 -0.1953125  -0.21508789 -0.2109375  -0.2109375  -0.2109375  -1.3671875
 -0.2109375  -0.28125   ]
[['s0-9', '0.17758994265764316', '0.219294308', '0.2707923252946056'], ['s10-19', '0.34280672223753605', '0.423309799', '0.522717829918331'], ['s30-39', '0.7092746820194112', '1.124597388', '1.7831163541543273'], ['s40-49', '0.742539113416158', '1.511734653', '1.9793905975326396'], ['s50-59', '0.8032179183923442', '1.560386081', '2.2758979656744183'], ['s60-69', '0.9617320373076483', '

In [7]:
with open('CovidStratified/Output/CIEstimates2.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(ci_header)

    # write multiple rows
    writer.writerows(ci_res)
    
f.close()

In [ ]:
starts = 20 #number of different starting points to test
trials = 50 # Number of trials per start point
iters = (starts * trials)
theta_iter = [[0] * 33] * iters
best_res = optimize.OptimizeResult(x = 100, fun = 100, success = False) #placeholder var to track best result

print ("Cumulative country calculation with randomized starting points:\n")
kwargs = dict(args = (prem, cases_dict, countries), method="L-BFGS-B")
for i in range(0, starts):
    theta_guess = np.random.gamma(4, 1/4, 14) # mean = 1, sd = 2
    for k in range(len(theta_guess)): theta_guess[k] = math.log(theta_guess[k], math.e)
    for j in range(0, trials):
        res = optimize.basinhopping(CovidEM2.Covid_KL2, theta_guess, minimizer_kwargs = kwargs)
        out_start = np.insert(np.insert(theta_guess, 2,1),10,1)
        out_end = np.insert(np.insert(res.x, 2,1),10,1)
        theta_iter[((i*trials)+j)] = np.append(np.append(out_start, out_end), res.fun)
        if (res.fun < best_res.fun): 
            best_res = res
            
    print(f"{((((i+1)*trials)/iters)*100):.2f}","%, ", end = "")


print(tabulate(theta_iter))


Cumulative country calculation with randomized starting points:

5.00 %, 10.00 %, 15.00 %, 